In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
from tensorflow.keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import tensorflow as tf
import keras
import random
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation, RandomTranslation, RandomFlip, RandomZoom
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import shutil
from tensorflow.keras.applications import Xception
import matplotlib.pyplot as plt


!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
fid = drive.ListFile({'q':"title='fish_data.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('fish_data.zip')
f.keys()
!unzip fish_data.zip

Archive:  fish_data.zip
   creating: fish data/train/
   creating: fish data/train/Bowal/
  inflating: fish data/train/Bowal/download - Copy.jpeg  
  inflating: fish data/train/Bowal/download (1) - Copy.jpeg  
  inflating: fish data/train/Bowal/download (1).jpeg  
  inflating: fish data/train/Bowal/download (3) - Copy.jpeg  
  inflating: fish data/train/Bowal/download (3).jpeg  
  inflating: fish data/train/Bowal/download (4) - Copy.jpeg  
  inflating: fish data/train/Bowal/download (4).jpeg  
  inflating: fish data/train/Bowal/download (5) - Copy.jpeg  
  inflating: fish data/train/Bowal/download (5).jpeg  
  inflating: fish data/train/Bowal/download.jpeg  
  inflating: fish data/train/Bowal/images (1) - Copy.jpeg  
  inflating: fish data/train/Bowal/images (1).jpeg  
  inflating: fish data/train/Bowal/images (10) - Copy.jpeg  
  inflating: fish data/train/Bowal/images (10).jpeg  
  inflating: fish data/train/Bowal/images (11) - Copy.jpeg  
  inflating: fish data/train/Bowal/images (1

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import Xception


In [ ]:
import cv2
import os
from tqdm import tqdm

folder_dir = '/content/fish data/train'
SIZE = (72,72) 
DOWNSAMPLE_RATIO = 4
JPEG_QUALITY = 60

# get the total number of files to process
total_files = sum(len(files) for _, _, files in os.walk(folder_dir))

# use tqdm to display a progress bar
with tqdm(total=total_files, desc="Processing Images") as pbar:
    for folder in os.listdir(folder_dir):
        for file in os.listdir(os.path.join(folder_dir, folder)):
            image_path = os.path.join(folder_dir, folder, file)
            # check if the file has read permission
            if not os.access(image_path, os.R_OK):
                print(f"No read permission for: {image_path}")
                continue
            # read the image
            img = cv2.imread(image_path)
            if img is None:
                print(f"Failed to read image: {image_path}")
                continue
            # check if the image is already in JPEG format
            if os.path.splitext(file)[-1].lower() in ['.jpg', '.jpeg']:
                img_resized = cv2.resize(img, SIZE)
                # write the resized image
                success = cv2.imwrite(image_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, JPEG_QUALITY])
                if not success:
                    print(f"Failed to write image: {image_path}")
            else:
                # convert the image to JPEG format
                new_image_path = os.path.splitext(image_path)[0] + '.jpg'
                img_resized = cv2.resize(img, SIZE)
                # write the resized image in JPEG format
                success = cv2.imwrite(new_image_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, JPEG_QUALITY])
                if not success:
                    print(f"Failed to write image: {new_image_path}")
                else:
                    # delete the original image and replace it with the new JPEG image
                    os.remove(image_path)
                    os.rename(new_image_path, image_path)
            pbar.update(1) # increment the progress bar


Processing Images: 100%|██████████| 1882/1882 [00:00<00:00, 2656.83it/s]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [ ]:
# Data preprocessing
train_data_dir = '/content/fish data/train'
validation_data_dir = '/content/fish data/validation'
batch_size = 32
img_height = 72
img_width = 72

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)

val_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


Found 1736 images belonging to 9 classes.
Found 438 images belonging to 9 classes.


In [ ]:
gpus = tf.config.list_logical_devices('GPU') 
stg=tf.distribute.MirroredStrategy(gpus)

In [ ]:
no_of_classes = 9

with stg.scope():
    img_augmentation = Sequential(
    [
        RandomRotation(factor=0.15),
        RandomTranslation(height_factor=0.1, width_factor=0.1),
        RandomFlip(),
        RandomZoom(height_factor=0.6)
    ],
    name="img_augmentation"
    )

    # Define base model
    base_model = Xception(weights=None, input_shape=(72, 72, 3), include_top=False)
    for layer in base_model.layers:
      layer.trainable = True
    # Add image augmentation as the first layer of base model
    inputs = Input(shape=(72, 72, 3))
    x = img_augmentation(inputs)
    outputs = base_model(x)

    # Add classification layers on top of the base model
    x = GlobalAveragePooling2D()(outputs)
    x = Dense(512, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(9, activation='softmax')(x)

    # Define the model
    model = Model(inputs=inputs, outputs=predictions)

    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(lr=0.001),
        metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 72, 72, 3)]       0         
                                                                 
 img_augmentation (Sequentia  (None, 72, 72, 3)        0         
 l)                                                              
                                                                 
 xception (Functional)       (None, 3, 3, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 128)               65664 

In [ ]:
# Train the model

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)


Epoch 1/50
54/54 [==============================] - 55s 151ms/step - loss: 2.1554 - accuracy: 0.1825 - val_loss: 2.1959 - val_accuracy: 0.1178
Epoch 2/50
54/54 [==============================] - 6s 112ms/step - loss: 2.0299 - accuracy: 0.2383 - val_loss: 2.1957 - val_accuracy: 0.1010
Epoch 3/50
54/54 [==============================] - 6s 117ms/step - loss: 1.9946 - accuracy: 0.2336 - val_loss: 2.1944 - val_accuracy: 0.1226
Epoch 4/50
54/54 [==============================] - 6s 106ms/step - loss: 1.9790 - accuracy: 0.2430 - val_loss: 2.1945 - val_accuracy: 0.0962
Epoch 5/50
54/54 [==============================] - 6s 112ms/step - loss: 1.9139 - accuracy: 0.2588 - val_loss: 2.1934 - val_accuracy: 0.0913
Epoch 6/50
54/54 [==============================] - 6s 114ms/step - loss: 1.9340 - accuracy: 0.2670 - val_loss: 2.1902 - val_accuracy: 0.1034
Epoch 7/50
54/54 [==============================] - 6s 116ms/step - loss: 1.9099 - accuracy: 0.2594 - val_loss: 2.1742 - val_accuracy: 0.1851
Epoch

In [ ]:
# Evaluate the model on the validation data
val_loss, val_acc = model.evaluate(validation_generator)
print("Validation accuracy:", val_acc)

14/14 [==============================] - 2s 101ms/step - loss: 1.7039 - accuracy: 0.4475
Validation accuracy: 0.44748857617378235
